In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
test.head()

In [ ]:
df.info()

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df.info()

In [ ]:
test['time'] = pd.to_datetime(test['time'])

In [ ]:
df['day'] = df['time'].dt.day

In [ ]:
test['day'] = test['time'].dt.day

In [ ]:
df.tail()

In [ ]:
df['hours'] = df['time'].dt.hour

In [ ]:
test['hours'] = test['time'].dt.hour

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
test['hours'].value_counts()

In [ ]:
df.info()

In [ ]:
df['hours'].value_counts()

In [ ]:
def AM_PM(x):
    if x >= 12:
        return 1
    else:
        return 0

In [ ]:
df['PM'] = df['hours'].apply(AM_PM)

In [ ]:
test['PM'] = test['hours'].apply(AM_PM)

In [ ]:
df.head()

In [ ]:
df['PM'].value_counts()

In [ ]:
test['PM'].value_counts()

In [ ]:
df.drop('time', axis = 1, inplace = True)
test.drop('time', axis = 1, inplace = True)

In [ ]:
df['direction'].value_counts()

In [ ]:
x = pd.get_dummies(df['direction'], drop_first = True)

In [ ]:
y = pd.get_dummies(test['direction'], drop_first = True)

In [ ]:
df = pd.concat([df, x], axis = 1)

In [ ]:
test = pd.concat([test, y], axis = 1)

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
df.shape

In [ ]:
test.shape

In [ ]:
df.drop(['row_id', 'direction'], axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('congestion', axis = 1)

In [ ]:
y = df['congestion']

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
test.drop('row_id', axis = 1, inplace = True)

In [ ]:
test.head()

In [ ]:
pred1 = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,pred1)
print(r2)

In [ ]:
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xgbr = xgboost.XGBRegressor(seed = 20)

In [ ]:
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   verbose=1)

In [ ]:
#clf.fit(X_train, y_train)

In [ ]:
#print("Best parameters:", clf.best_params_)
#print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

In [ ]:
test.drop('direction', axis = 1, inplace = True)

In [ ]:
a = model.predict(test)

In [ ]:
data = pd.DataFrame(a, columns = ['congestion'])

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
submit.head()

In [ ]:
submit.drop('congestion', axis = 1, inplace = True)

In [ ]:
submit = pd.concat([submit, data], axis = 1)

In [ ]:
submit.head()

In [ ]:
submit.to_csv('submit.csv', index = False)